## Restraunt AI assistant -
 Available tools:
 - get_menu()
 - get_available_tables(date,people)
 - book_table(name, date, time, people)
 - calculate_bill(items)

In [70]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from datetime import datetime
import psycopg

In [68]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("Got the key")
openai = OpenAI()
MODEL = "gpt-4.1-mini"

Got the key


In [52]:
current_date =  datetime.now().strftime('%Y-%m-%d')
current_time = datetime.now().strftime('%H:%M:%S')
print(f"Current date - {current_date} and current time is {current_time}")
print(datetime.now())

Current date - 2026-02-27 and current time is 15:58:50
2026-02-27 15:58:50.155665


In [60]:
def get_menu():
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT * FROM Menu;")
                rows = cur.fetchall()
    return rows

def get_available_tables(date, time, people):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT t.*
                FROM tables t
                WHERE t."Capacity" >= %s
                AND NOT EXISTS (
                    SELECT 1
                    FROM bookings b
                    WHERE b.table_id = t.id
                    AND b."Date" = %s
                    AND (
                        b."Time" < (%s::time + interval '2 hours')
                        AND
                        (b."Time" + interval '2 hours') > %s::time
                    )
                )
            """,(people,date,time,time))
            rows = cur.fetchall()
    return rows

def book_table(name,date,time,people):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            available_tables = get_available_tables(date,time, people)
            if available_tables:
                table_id = available_tables[0][0]
                cur.execute("""
                    INSERT INTO bookings("table_id", "Name", "Date", "Time", "People")
                    VALUES (%s, %s, %s, %s, %s) RETURNING booking_id
                """, (table_id, name, date, time, people))
                booking_id = cur.fetchone()[0]
                conn.commit()
                return f"Table {table_id} booked successfully with booking id - {booking_id} for {name} on {date} at {time}."
            else:
                return "No available tables for the selected date."

def add_to_cart(booking_id, items):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            for item, quantity in items.items():
                print(item, quantity)
                cur.execute("INSERT INTO cart(\"booking_id\", \"item\", \"quantity\") VALUES (%s, %s, %s)", (booking_id, item, quantity))
            conn.commit()
            return "Your items are added"

def remove_or_update_cart(booking_id, item, quantity=0):
     with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            cur.execute("UPDATE cart SET \"quantity\" = %s WHERE booking_id = %s AND item = %s", (quantity,booking_id, item))
            conn.commit()
            return f"Item quantity for {item} for booking id {booking_id} is udpated to {quantity}"

def calculate_bill(booking_id):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            total_price = 0
            cur.execute("SELECT item, quantity from cart where booking_id=%s", (booking_id,))
            rows = cur.fetchall()
            for items in rows:
                print(items)
                cur.execute("SELECT \"Price\" from menu where \"Dish Name\"= %s", (items[0],))
                total_price += (items[1] * (cur.fetchone()[0]))
            return total_price

def cancel_booking(booking_id):
    with psycopg.connect("dbname=MyRestaurantDB user=postgres password=Veenamaa@2103") as conn:
        with conn.cursor() as cur:
            now = datetime.now()
            cur.execute('SELECT "Date", "Time" from bookings where booking_id=%s',(booking_id,))
            rows = cur.fetchone()
            if rows: 
                date,time = rows
                booking_datetime = datetime.combine(date, time)
                if booking_datetime <= now:
                    return "Too late to cancel"
                else:
                    cur.execute("DELETE FROM bookings where booking_id=%s",(booking_id,))
                    cur.execute("DELETE FROM cart where booking_id=%s", (booking_id,))
                    conn.commit()
                    return f"Booking for booking id {booking_id} canceled successfully"
            else:
                return "No bookings for the booking id found"

In [61]:
get_menu_function = {
    "name": "get_menu",
    "description":"Retrieve all available dishes from the restaurant menu along with their prices.",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}
get_available_tables_function = {
    "name": "get_available_tables",
    "description":"Retrieve available tables for a given date and time with capacity greater than or equal to the specified number of people. Tables booked within a 2-hour overlapping window should not be returned.",
    "parameters":{
        "type":"object",
        "properties":{
            "date":{
                "type": "string",
                "description": "Date in YYYY-MM-DD format"
            },
            "time": {
                "type": "string",
                "description": "Time in HH:MM:SS format (24-hour format)"
            },
            "people":{
                "type":"integer",
                "description":"Number of people for table booking"
            }
        },
        "required": ["date", "time", "people"]
    }
}

book_table_function = {
    "name": "book_table",
    "description":"Book a table for a specific date and time with name and number of people. Returns booking_id",
    "parameters":{
        "type":"object",
        "properties":{
            "name":{
                "type": "string",
                "description": "Name of the person for booking the table."
            },
            "date":{
                "type": "string",
                "description": "Date in YYYY-MM-DD format"
            },
            "time": {
                "type": "string",
                "description": "Time in HH:MM:SS format (24-hour format)"
            },
            "people":{
                "type":"integer",
                "description":"Number of people for table booking"
            }
        },
        "required": ["name","date", "time", "people"]
    }
}
add_to_cart_function = {
    "name": "add_to_cart",
    "description": "Adds items to the cart for a specific booking. Items should be a dictionary where keys are dish names and values are quantities.",
    "parameters": {
        "type": "object",
        "properties": {
            "booking_id": {
                "type": "integer",
                "description": "Booking ID for the reservation"
            },
            "items": {
                "type": "object",
                "description": "Dictionary where keys are dish names and values are quantities",
                "additionalProperties": {
                    "type": "integer",
                    "description": "Quantity of the dish"
                }
            }
        },
        "required": ["booking_id", "items"]
    }
}

remove_from_cart_function = {
    "name": "remove_or_update_cart",
    "description": "Updates items from cart for booking id to a specific quantity. If quantity of item is not given as parameter then it's quantity changes to 0",
    "parameters": {
        "type": "object",
        "properties": {
            "booking_id": {
                "type": "integer",
                "description": "Booking ID for the reservation"
            },
            "item": {
                "type": "string",
                "description": "Item name for which quantity is to be altered or item name which has to be removed"
            },
            "quantity": {
                "type": "integer",
                "description": "New quantity for the item. If not specified, then item quantity will default to 0. (Optional)"
            }
        },
        "required": ["booking_id", "item"]
    }
}

calculate_bill_function = {
    "name":"calculate_bill",
    "description":"Calculates bill using booking id, gets items from cart and adds amount from menu to return the bill amount",
    "parameters":{
        "type":"object",
        "properties":{
            "booking_id":{
                "type":"integer",
                "description":"Booking id for calculating bill"
            }
        },
        "required":["booking_id"]
    }
}

cancel_booking_function= {
    "name":"cancel_booking",
    "description":"Cancels booking using booking id. Removes cart items and booked table. Returns error if user tries to cancel booking after the booked date and time. Bookings can be cancelled only before the booked date and time. ",
    "parameters":{
        "type":"object",
        "properties":{
            "booking_id":{
                "type":"integer",
                "description":"Booking id for cancellation"
            }
        },
        "required":["booking_id"]
    }
}

In [63]:
tools = [{"type": "function", "function": get_menu_function},
         {"type": "function", "function": get_available_tables_function},
         {"type": "function", "function": book_table_function},
         {"type": "function", "function": add_to_cart_function},
         {"type": "function", "function": remove_from_cart_function},
         {"type": "function", "function": calculate_bill_function},
         {"type": "function", "function": cancel_booking_function}
        ]

In [64]:
available_functions = {"get_menu" : get_menu,
                      "get_available_tables" : get_available_tables,
                       "book_table" : book_table,
                       "add_to_cart" : add_to_cart,
                       "remove_or_update_cart" : remove_or_update_cart,
                       "calculate_bill" : calculate_bill,
                       "cancel_booking" : cancel_booking
                      }

In [66]:
system_prompt = f"""
You are a polite and intelligent Restaurant Assistant.

You must ALWAYS use the provided tools when retrieving or booking or cancelling:
- Menu items and prices
- Table availability
- Booking confirmation
- Cart updates
- Bill calculation
- Booking cancellation

Never hallucinate prices, availability, booking IDs, or cart data.

1️ Menu
- When user asks for menu → call get_menu_function.
- When user asks for suggestions → fetch menu and suggest 5 random items.
- Never invent prices.

2️ Table Availability
- Use get_available_tables_function.
- Required: date, time, people.
- If missing info → ask user before calling tool.
- If available → ask user if they want to book.
- If unavailable → offer alternate times:
  - 2 hours before
  - 2 hours after
  - 4 hours after

3️ Booking
- Use book_table_function.
- Required: name, date, time, people.
- Store latest booking_id for future cart actions.

4️ Cart Management
- Use add_to_cart_function.
- Use remove_or_update_cart_function.
- Always confirm booking_id.
- Ask follow-up: "Would you like to add or modify anything else?"

5️ Calculate Bill
- Use calculate_bill_function.
- Return total amount clearly.

6️ Cancel Booking
- Use cancel_booking_function.
- If response = "Too late to cancel":
  Tell user to contact support at 8888888888.


General Rules:
- Be conversational.
- Ask for missing information.
- Never assume.
- Always rely on tools for data.
"""

In [81]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        print("Called tool - ", function_name)
        arguments = json.loads(tool_call.function.arguments)
        function_to_call = available_functions.get(function_name)
        if function_to_call:
            result = function_to_call(**arguments)
            responses.append({
                "role": "tool",
                "content": str(result),
                "tool_call_id": tool_call.id
            })
    return responses

In [82]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]}  for h in history]
    messages = [{"role":"system","content":system_prompt}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL, messages = messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages = messages)

    return response.choices[0].message.content

In [83]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Called tool -  get_menu
Called tool -  get_available_tables
Called tool -  book_table
Called tool -  add_to_cart
Pav bhaji 2
Called tool -  calculate_bill
('Pav bhaji', 2)
Called tool -  get_available_tables
Called tool -  book_table
Called tool -  get_available_tables
Called tool -  get_available_tables
Called tool -  book_table
Called tool -  get_available_tables
Called tool -  book_table
